In [ ]:
### import modules
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.patheffects as pe
import matplotlib.transforms as transforms
from matplotlib.gridspec import GridSpec
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.ndimage import gaussian_filter
from scipy import ndimage
from scipy.interpolate import interp1d
from scipy import interpolate
from tenet.util import sphMap
import scipy.stats
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import ks_2samp, anderson_ksamp
from scipy.optimize import curve_fit
import os
import time
import h5py
import rohr_utils as ru 
import random
import six
%matplotlib widget

plt.style.use('fullpage.mplstyle')

zs, times = ru.return_zs_costimes()
times /= 1.0e9 # [Gyr]
scales = 1. / (1.+ zs)

os.chdir('/u/reric/Scripts/')
! pwd


In [ ]:
# define some plotting parameters
figsizewidth  = 6.902 # the textwidth in inches of MNRAS
figsizeratio = 9. / 16.
figsizeheight = figsizewidth * figsizeratio

figsizewidth_column = (244. / 508.) * figsizewidth
figsizeheight_column = figsizewidth_column * figsizeratio

outdirec_figures = '/u/reric/Figures/ColdPast/TNG-Cluster/'
outdirec_overleaf = '/u/reric/Papers/Rohretal_TNG_CoolerPast/figures/'
outdirecs = [outdirec_figures, outdirec_overleaf]
savefig = False

In [ ]:
# load the gas radial profile dictionary
grp_keys = ['SnapNum', 'SubfindID', 'CosmicTime', 'HostGroup_M_Crit200',
            'HostGroup_R_Crit200', 'SubhaloMass', 'HostSubhaloGrNr',
            'Subhalo_Mstar_Rgal', 'Subhalo_Rgal',
            'SubhaloColdGasMass', 'SubhaloHotGasMass', 'SubhaloGasMass',
            'radii', 'SubhaloColdGasMassShells', 'vol_shells', 'SubhaloColdGasDensityShells',
            'SubhaloCGMColdGasMass', 'SubhaloCGMColdGasFraction',
            'CGMTemperaturesHistogram', 'CGMTemperaturesHistogramBincents',
            'NSatellites_total', 'Redshift', 'SubhaloCoolCGMFraction']

# plus some extra datasets that are only relevant for the fiducial dictionary
fiducial_keys = ['MainBHMass', 'MainBHParticleID', 'MainBH_CumEgyInjection_RM', 'MainBH_RM_FirstSnap',
                 'NSatellites_Mstar>1.0e10_fgas>0.01_dsathost<R200c', 'NSatellites_Mstar>1.0e7_SF_dsathost<R200c',
                 'NSatellites_Mstar>1.0e7_fgas>0.01_dsathost<R200c', 'NSatellites_Mstar>1.0e9_fgas>0.01_dsathost<R200c',
                 'NSatellties_Mstar>1.0e7_dsathost<R200c', 'NSatellties_Mstar>1.0e9_dsathost<R200c',
                 'NSatellites_dsathost<R200c']

def load_grpdict(infname, sim='L680n8192TNG', keys=None):
    result = {}
    with h5py.File('../Output/%s_subfindGRP/'%sim + infname, 'r') as f:
        for group_key in f.keys():
            result[group_key] = {}
            if not keys:
                keys = f[group_key].keys()
            for dset_key in keys:
                if 'xray' in dset_key:
                    continue
                if dset_key not in f[group_key]:
                    continue
                result[group_key][dset_key] = f[group_key][dset_key][:]
        f.close()
    
    return result

# NB: the output file and directory are at L680n8192TNG, while all figures are saved as 'TNG-Cluster'
sim = 'L680n8192TNG'

# fiducial method
infname = 'central_subfind_%s_branches.hdf5'%sim
TNGCluster_grp_dict = load_grpdict(infname, sim, keys=grp_keys + fiducial_keys)
TNGCluster_grp_dict_keys = list(TNGCluster_grp_dict.keys())

In [ ]:
sim = 'TNG-Cluster'
basePath = ru.ret_basePath(sim)
gas_ptn = 0
redshift = 4.
snapNum = 21
haloID =  14131544
header = ru.loadHeader(basePath, snapNum)

h = header['HubbleParam']
a = header['Time']
boxsize = header['BoxSize'] * a / h

mass_hilim = 1.0e8
boxsizeimg = 3. # r200c
depth = 3.0 # r200c
nPixels = 1024
gas_hsml_fact = 1.5
axes = [0,1]


In [ ]:

halo = il.groupcat.loadSingle(basePath, snapNum, haloID=haloID)
HaloPos = halo['GroupPos'] * a / h
R200c = halo['Group_R_Crit200'] * a / h

_gas_cells = il.snapshot.loadOriginalZoom(basePath, snapNum, haloID, gas_ptn)
_gas_cells = ru.calc_temp_dict(_gas_cells)

Masses = _gas_cells['Masses'] * 1.0e10 / h
_Coordinates = _gas_cells['Coordinates'] * a / h
Coordinates = ru.shift(_Coordinates, HaloPos, boxsize)


In [ ]:
mass_mask = Masses < 1.0e8
depth_mask = np.abs(Coordinates[:,2]) < (depth * R200c / 2.)
mask = mass_mask & depth_mask

gas_cells = {}
for key in _gas_cells:
    if key == 'count':
        gas_cells[key] = mask[mask].size
    else:
        gas_cells[key] = _gas_cells[key][mask].copy()



In [ ]:
Masses = gas_cells['Masses'] * 1.0e10 / h
Coordinates = gas_cells['Coordinates'] * a / h
Densities = gas_cells['Density'] * 1.0e10 / h / (a / h)**3
Sizes = (Masses / (Densities * 4./3. * np.pi))**(1./3.) * gas_hsml_fact

pos = Coordinates[:,axes]
hsml = Sizes
mass = Masses
boxSizeImg = [boxsizeimg*R200c, boxsizeimg*R200c] # kpc
boxSizeSim = [boxsize, boxsize, boxsize]
boxCen = HaloPos[axes]    
ndims = 3
colDens = False
quant = np.log10(gas_cells['Temperature'])

result = sphMap.sphMap(pos, hsml, mass, quant, [0,1], boxSizeImg, boxSizeSim, boxCen, [nPixels, nPixels], ndims, colDens=colDens)


In [ ]:
fig, ax = plt.subplots(figsize=(figsizewidth, figsizewidth*1.15))

extent = [-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[0]/2., boxSizeImg[0]/2.]
length = 50 #pkpc

imshow_kwargs = dict(cmap='turbo', norm=mpl.colors.Normalize(vmin=3.25, vmax=6.5),
                     origin='lower', extent=extent)
img = ax.imshow(result, **imshow_kwargs)

ax.set_xticks([])
ax.set_yticks([])

color='black'
circ_kwargs = dict(color=color, fill=False)
circle_r200c = plt.Circle((0., 0.), R200c, **circ_kwargs)
ax.add_patch(circle_r200c)
circle_015r200c = plt.Circle((0., 0.), 0.15*R200c, **circ_kwargs)
ax.add_patch(circle_015r200c)

x0 = -boxSizeImg[0]/2. + (boxSizeImg[0]/2. * 0.075)# kpc
y0 = boxSizeImg[0]/2. -(boxSizeImg[0]/2. * 0.1)# kpc
ax.plot([x0, x0+length], [y0, y0], color=color, marker='None', ls='-')
text_kwargs = dict(fontsize=12, color=color)
ax.text(x0+length/2., y0-(boxSizeImg[0]/2. * 0.025), '%d pkpc'%(int(length)), ha='center', va='top', **text_kwargs)
ax.text(x0+length/2., y0+(boxSizeImg[0]/2. * 0.025), '%d ckpc'%(int(round(length / a))), ha='center', va='bottom', **text_kwargs)

text = (r'TNG-Cluster $z=%.1f$'%redshift + '\n'
        'haloID: %d '%haloID + r'$\hat{z}$ proj.')
ax.text(.99, 0.99, text, ha='right', va='top', ma='right', transform=ax.transAxes, **text_kwargs)

# add colorbar 
cbar = fig.colorbar(img, ax=ax, orientation='horizontal', label=r'Gas Temperature [log K]', extend='both')

fname = 'TNG-Cluster_snapNum%03d_haloID%08d_TemperatureMap.pdf'%(snapNum, haloID)
if savefig:
    for outdirec in outdirecs:
        fig.savefig(outdirec + fname, bbox_inches='tight', dpi=300)